## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

import warnings
warnings.filterwarnings("ignore")

`(3) langfuase handler 설정`

In [ ]:
from langfuse.langchain import CallbackHandler

# LangChain 콜백 핸들러 생성
langfuse_handler = CallbackHandler()

`(4) Langfuse 클라이언트 설정`

In [ ]:
from langfuse import get_client

# Langfuse 클라이언트 초기화
langfuse = get_client()

# 연결 테스트
assert langfuse.auth_check()

---

## **프롬프트 관리**

- Langfuse는 **프롬프트 CMS** 기능 제공

- **버전 관리** 기능으로 롤백과 성능 비교 가능

### 1. **프롬프트 생성**

In [ ]:
# 텍스트 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic",  # 프롬프트 이름
    type="text",          
    prompt="{{criticLevel}} 영화 평론가로서, {{movie}}를 어떻게 생각하시나요?",
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "text"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7
    }
)

In [ ]:
# 챗 프롬프트 생성
langfuse.create_prompt(
    name="movie-critic-chat",  # 프롬프트 이름
    type="chat",          
    prompt=[
        {
            "role": "system",
            "content": "당신은 {{criticLevel}} 영화 평론가입니다."
        },
        {
            "role": "user",
            "content": "영화 평론가로서, {{movie}}를 어떻게 생각하시나요?"
        }
    ], #type: ignore
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa", "chat"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7
    }
)  

### **[실습]**

- text 기반 프롬프트와 chat 기반 프롬프트를 각각 구현하고, Langfuse UI에서 확인하세요. 

In [ ]:
# 텍스트 프롬프트 생성

In [ ]:
# 챗 프롬프트 생성


### 2. **프롬프트 활용**

In [ ]:
# 프로덕션 버전 가져오기
prompt = langfuse.get_prompt("movie-critic")

# 프롬프트 출력
print(f"모델: {prompt.config['model']}")
print(f"온도: {prompt.config['temperature']}")
print(f"라벨: {prompt.labels}")
print(f"태그: {prompt.tags}")
print(f"프롬프트: {prompt.prompt}")
print("-" * 100)

# 랭체인 프롬프트 출력
print(f"프롬프트: {prompt.get_langchain_prompt()}")

### **[실습]**

- "movie-critic-chat" 프롬프트를 Langfuse에서 가져와서 내용을 출력합니다.

In [ ]:
# chat 프롬프트 가져오기

In [ ]:
# 새로운 버전 생성
langfuse.create_prompt(
    name="movie-critic",  # 같은 이름 사용
    type="text",          
    prompt="당신은 {{criticLevel}} 영화 평론가입니다.\n\n영화 {{movie}}를 어떻게 생각하시나요?",
    labels=["production"],       # 프로덕션 레이블
    tags=["movie", "qa"],    # 태그
    config={
        "model": "gpt-4.1-mini",
        "temperature": 0.7
    }
)

In [ ]:
# 특정 버전 가져오기
prompt_v1 = langfuse.get_prompt("movie-critic", version=1)

# 프롬프트 출력
print(f"모델: {prompt_v1.config['model']}")
print(f"온도: {prompt_v1.config['temperature']}")
print(f"라벨: {prompt_v1.labels}")
print(f"프롬프트: {prompt_v1.prompt}")

### **[실습]**

- "movie-critic-chat" 프롬프트를 수정하고, labels 속성은 "staging"으로 커스텀 지정하세요.
- Langfuse에서 "staging" 버전을 가져와서 내용을 출력합니다.

In [ ]:
# staging 버전 생성


In [ ]:
# staging 버전 가져오기

In [ ]:
from langchain_core.prompts import PromptTemplate

# Langchain과 통합 - 'text' 프롬프트
langchain_prompt = PromptTemplate.from_template(
    prompt_v1.get_langchain_prompt(),
    metadata={"langfuse_prompt": prompt_v1},
)

print(langchain_prompt.format(criticLevel="비평가", movie="인셉션"))

In [ ]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화 (프롬프트 설정에서 가져온 값 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-mini"),
    temperature=prompt.config.get("temperature", 0.7)
)

# 체인 생성 및 실행
chain = langchain_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}  # 콜백 핸들러 추가
)

# 응답 출력
print(response.content)

### **[실습]**

- 앞에서 직접 정의한 text 기반 프롬프트를 가져와서, LangChain과 통합하여 Tracing 추적을 실행합니다. 
- 실행 결과를 Langfuse UI에서 확인하세요. 

In [ ]:
# 여기에 코드를 추가하세요.

In [ ]:
# 특정 라벨 가져오기
prompt_staging = langfuse.get_prompt("movie-critic", label="latest")  # production, latest

# 프롬프트 출력
print(f"모델: {prompt_staging.config['model']}")
print(f"온도: {prompt_staging.config['temperature']}")
print(f"라벨: {prompt_staging.labels}")
print(f"프롬프트: {prompt_staging.prompt}")

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

# Langchain과 통합 - 'chat' 프롬프트
langchain_prompt = ChatPromptTemplate.from_template(
    prompt_staging.get_langchain_prompt(type="chat"),
)
langchain_prompt.metadata = {"langfuse_prompt": prompt_staging}

print(langchain_prompt.format(criticLevel="비평가", movie="인셉션"))

In [ ]:
from langchain_openai import ChatOpenAI

# ChatOpenAI 모델 초기화 (프롬프트 설정에서 가져온 값 사용)
model = ChatOpenAI(
    model=prompt.config.get("model", "gpt-4.1-mini"),
    temperature=prompt.config.get("temperature", 0.7)
)

# 체인 생성 및 실행
chain = langchain_prompt | model
response = chain.invoke(
    input={"criticLevel": "전문가", "movie": "인셉션"},
    config={"callbacks": [langfuse_handler]}  # 콜백 핸들러 추가
)

# 응답 출력
print(response.content)

### **[실습]**

- 앞에서 직접 정의한 chat 기반 프롬프트를 가져와서, LangChain과 통합하여 Tracing 추적을 실행합니다. 
- 실행 결과를 Langfuse UI에서 확인하세요. 

In [ ]:
# 여기에 코드를 추가하세요.